## **DAT 402 Project 1**   
## **Predicting Spotify Song Popularity**

**Team Members:**
- Pete VanBenthuysen
- Emily Szolnoki

**Project Goal:**
Predict whether a song will be popular (popularity $ \geq 70 $) based on its musical features, such as **danceability**, **energy**, and **tempo**. This project helps us identify which characteristics are most influential in determining whether a song becomes popular. We are most interested in analyzing and building a model to predict song popularity based on those musical attributes and more, allowing for potential insights in playlist curation and music production.

**Dataset:**
We are using a dataset from Kaggle called 'Spotify Tracks DB'. The raw dataset includes 18 columns and 232,725 rows with a mix of categorical and numerical features. There are also no missing values.

 (https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db)

**Columns:**
- **genre**: Genre of the song.
- **artist_name**: Name of the artist.
- **track_name**: Name of the track.
- **track_id**: Unique ID for each track.
- **popularity**: Popularity score of the song (ranging from 0 to 100).
- **acousticness**: A measure of how acoustic the song is (ranging from 0 to 1).
- **danceability**: How suitable the song is for dancing (ranging from 0 to 1).
- **duration_ms**: Duration of the trach in milliseconds.
- **energy**: A measure of energy, typically related to the intensity and activity of the track (ranging from 0 to 1).
- **instrumentalness**: A measure of whether the track is instrumental (ranging from 0 to 1).
- **key**: The musical key of the track.
- **liveness**: Measure of the presence of a live audience (ranging from 0 to 1).
- **loudness**: The overall loudness of the track in decibels.
- **mode**: The modality of the track.
- **speechiness**: Speechiness measure of the track (ranging from 0 to 1).
- **tempo**: The tempo (beats per minute) of the track.
- **time_signature**: The time signature of the track.
- **valence**: A measure of musical positiveness (ranging from 0 to 1).

### **Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Model selection and evaluation
from sklearn.model_selection import train_test_split as tts, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, r2_score

# Preprocessing
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'pandas'

### **Data Loading and Cleaning**

In [ ]:
# Load dataset
df = pd.read_csv("data/SpotifyFeatures.csv", parse_dates=False) # Set parse_dates=False to avoid parsing dates
print(df.head())

print(df.isnull().sum()) # Check for missing values